In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip -q install segmentation-models-pytorch==0.3.3 albumentations==1.4.7 opencv-python==4.10.0.84 tqdm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.8/58.8 kB 7.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.5/68.5 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.7/106.7 kB 11.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 155.7/155.7 kB 18.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 MB 15.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 105.0 MB/s eta 0:00:00


In [ ]:
import wandb
wandb.login()

wandb: WARNING Calling wandb.login() after wandb.init() has no effect.


False

In [ ]:
import os
import cv2
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import albumentations as A
import segmentation_models_pytorch as smp
from tqdm import tqdm
import wandb

# ===============================
# 🔹 EDIT THIS PATH
# ===============================
BASE = "/content/drive/MyDrive/Crack segmentation/1500"

TRAIN_IMG = os.path.join(BASE, "Train", "Images")
TRAIN_MSK = os.path.join(BASE, "Train", "Masks")
VAL_IMG   = os.path.join(BASE, "Valid", "Images")
VAL_MSK   = os.path.join(BASE, "Valid", "Masks")
TEST_IMG  = os.path.join(BASE, "Test", "Images")
TEST_MSK  = os.path.join(BASE, "Test", "Masks")

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

# ===============================
# 🔹 Hyperparameters
# ===============================
IMG_SIZE = 512
BATCH = 6
EPOCHS = 60
LR = 1e-4

# ===============================
# 🔹 Initialize W&B
# ===============================
wandb.init(
    project="crack-segmentation",
    name="unet_resnet34_1500",
    config={
        "epochs": EPOCHS,
        "batch_size": BATCH,
        "lr": LR,
        "img_size": IMG_SIZE,
        "model": "Unet_resnet34",
        "loss": "BCE + Dice"
    }
)

config = wandb.config

In [ ]:
def list_files(folder):
    exts = (".jpg", ".jpeg", ".png", ".bmp", ".tif", ".tiff")
    return sorted([f for f in os.listdir(folder) if f.lower().endswith(exts)])

class CrackDataset(Dataset):
    def __init__(self, img_dir, msk_dir, size=512, augment=False):
        self.img_dir = img_dir
        self.msk_dir = msk_dir
        self.files = list_files(img_dir)
        self.size = size

        if augment:
            self.tf = A.Compose([
            A.Resize(size, size),
            A.HorizontalFlip(p=0.5),
            A.VerticalFlip(p=0.5),
            A.ShiftScaleRotate(shift_limit=0.05, scale_limit=0.10, rotate_limit=15, p=0.7),
            A.RandomBrightnessContrast(p=0.4),
            A.GaussNoise(var_limit=(5.0, 30.0), p=0.3),
            A.MotionBlur(blur_limit=3, p=0.2),
])
        else:
            self.tf = A.Compose([A.Resize(size, size)])

    def __len__(self):
        return len(self.files)

    def __getitem__(self, idx):
        name = self.files[idx]
        img_path = os.path.join(self.img_dir, name)

        img = cv2.imread(img_path)
        if img is None:
            raise FileNotFoundError(f"Image not readable: {img_path}")
        img = img[:, :, ::-1]  # BGR->RGB

        base = os.path.splitext(name)[0]

        # ✅ Try multiple mask extensions
        mask_candidates = [base + e for e in [".png", ".jpg", ".jpeg", ".bmp", ".tif", ".tiff"]]
        mask_path = None
        for mc in mask_candidates:
            p = os.path.join(self.msk_dir, mc)
            if os.path.exists(p):
                mask_path = p
                break

        if mask_path is None:
            raise FileNotFoundError(f"Mask not found for image {name}. Tried: {mask_candidates}")

        mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)
        if mask is None:
            raise FileNotFoundError(f"Mask not readable: {mask_path}")

        # ✅ Convert mask to binary 0/1
        mask = (mask > 127).astype(np.float32)

        aug = self.tf(image=img, mask=mask)
        img, mask = aug["image"], aug["mask"]

        img = img.astype(np.float32) / 255.0
        mean = np.array([0.485, 0.456, 0.406], dtype=np.float32)
        std  = np.array([0.229, 0.224, 0.225], dtype=np.float32)
        img = (img - mean) / std

        img = np.transpose(img, (2, 0, 1))        # CHW
        mask = np.expand_dims(mask, axis=0)       # 1,H,W

        return torch.from_numpy(img).float(), torch.from_numpy(mask).float(), name

In [ ]:
model = smp.Unet(
    encoder_name="resnet34",
    encoder_weights="imagenet",
    in_channels=3,
    classes=1
).to(DEVICE)

# ✅ Handles class imbalance (thin cracks)
pos_weight = torch.tensor([5.0], device=DEVICE)  # try 2.0~6.0 later
bce = nn.BCEWithLogitsLoss(pos_weight=pos_weight)

def dice_score(logits, targets):
    probs = torch.sigmoid(logits)
    num = 2 * (probs * targets).sum(dim=(2,3))
    den = (probs + targets).sum(dim=(2,3)) + 1e-6
    return (num / den).mean().item()

def total_loss(logits, targets):
    return bce(logits, targets) + dice_loss(logits, targets)

def dice_score(logits, targets):
    probs = torch.sigmoid(logits)
    preds = (probs > 0.5).float()
    num = 2 * (preds * targets).sum(dim=(2,3))
    den = (preds + targets).sum(dim=(2,3)) + 1e-6
    return (num / den).mean().item()

In [ ]:
train_ds = CrackDataset(TRAIN_IMG, TRAIN_MSK, IMG_SIZE, True)
val_ds   = CrackDataset(VAL_IMG, VAL_MSK, IMG_SIZE, False)

train_loader = DataLoader(train_ds, batch_size=BATCH, shuffle=True)
val_loader   = DataLoader(val_ds, batch_size=BATCH, shuffle=False)

optimizer = torch.optim.AdamW(model.parameters(), lr=LR, weight_decay=1e-4)

# ✅ Auto-reduce LR when val_dice stops improving
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(
    optimizer,
    T_max=EPOCHS,
    eta_min=1e-6
)

best_dice = 0.0
best_epoch = -1

patience = 12
no_improve = 0

for epoch in range(EPOCHS):
    model.train()
    train_loss = 0.0

    for imgs, masks, _ in tqdm(train_loader, desc=f"Epoch {epoch+1}/{EPOCHS} [train]"):
        imgs, masks = imgs.to(DEVICE), masks.to(DEVICE)

        optimizer.zero_grad()
        logits = model(imgs)
        loss = total_loss(logits, masks)
        loss.backward()
        optimizer.step()

        train_loss += loss.item()

    model.eval()
    val_loss = 0.0
    val_dice = 0.0

    with torch.no_grad():
        for imgs, masks, _ in tqdm(val_loader, desc=f"Epoch {epoch+1}/{EPOCHS} [val]"):
            imgs, masks = imgs.to(DEVICE), masks.to(DEVICE)
            logits = model(imgs)

            val_loss += total_loss(logits, masks).item()
            val_dice += dice_score(logits, masks)

    val_loss = val_loss / len(val_loader)
    val_dice = val_dice / len(val_loader)
    train_loss = train_loss / len(train_loader)

    # ✅ Step scheduler based on validation dice (MUST be inside loop)
    scheduler.step()

    # ✅ Log to W&B
    wandb.log({
        "epoch": epoch + 1,
        "train_loss": train_loss,
        "val_loss": val_loss,
        "val_dice": val_dice,
        "lr": optimizer.param_groups[0]["lr"],
        "best_val_dice": best_dice
    })

    # ✅ Save best model + early stopping
    if val_dice > best_dice:
        best_dice = val_dice
        best_epoch = epoch + 1
        no_improve = 0
        torch.save(model.state_dict(), "best_unet.pth")
        wandb.save("best_unet.pth")
        print(f"✅ New best model saved at epoch {best_epoch} | best_dice={best_dice:.4f}")
    else:
        no_improve += 1
        print(f"no_improve={no_improve}/{patience} | best_epoch={best_epoch} | best_dice={best_dice:.4f}")

        if no_improve >= patience:
            print("⛔ Early stopping triggered (no val_dice improvement).")
            break

Epoch 1/60 [val]: 100%|██████████| 50/50 [00:11<00:00,  4.17it/s]
wandb: WARNING Symlinked 1 file into the W&B run directory; call wandb.save again to sync new files.


✅ New best model saved at epoch 1 | best_dice=0.3548


Epoch 2/60 [val]: 100%|██████████| 50/50 [00:11<00:00,  4.21it/s]


✅ New best model saved at epoch 2 | best_dice=0.4495


Epoch 3/60 [val]: 100%|██████████| 50/50 [00:11<00:00,  4.35it/s]


...

no_improve=10/12 | best_epoch=31 | best_dice=0.5632


Epoch 42/60 [val]: 100%|██████████| 50/50 [00:12<00:00,  4.03it/s]


no_improve=11/12 | best_epoch=31 | best_dice=0.5632


Epoch 43/60 [val]: 100%|██████████| 50/50 [00:11<00:00,  4.17it/s]

no_improve=12/12 | best_epoch=31 | best_dice=0.5632
⛔ Early stopping triggered (no val_dice improvement).
